In [ ]:
#Oppgaver
#1 - Hva er (ca) accuracy etter 0 epoker? Hvorfor?
#2 - Hvor god accuracy klarer du å få med "fully-connected" og "convolutional" nettverk?
### Du kan justere konstantene n_epochs, batch_size, learning_rate, model_dimension.
###Det er ikke nødvendig å trene mer enn kanskje 5 minutter.
#3 - Plot eksempler på feil-klassifikasjon for et godt ConvNet. Hva slags feil gjør nettverket?

#Åpent spørsmål: hva er svakhetene eller begrensningene til et slikt nettverk?

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

In [ ]:
n_epochs = 3
batch_size = 64
learning_rate = 0.01
momentum = 0.5
model_dimension = 8

torch.backends.cudnn.enabled = False

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self, m_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, m_dim)
        self.fc2 = nn.Linear(m_dim, m_dim)
        self.fc3 = nn.Linear(m_dim, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
network = Net(model_dimension)
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
def train():
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()


In [ ]:
def test(epoch):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  print('\nTest set, {:.0f} epochs: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    epoch, test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
def debug(n = 0):
  network.eval()
  with torch.no_grad():
    for ix,(data, target) in enumerate(test_loader):
      output = network(data)
      pred = output.data.max(1, keepdim=False)[1]
      if ix == n: break
  fig = plt.figure()
  for i in range(5*5):
    plt.subplot(5,5,i+1)
    plt.tight_layout()
    plt.imshow(data[i][0], cmap='gray', interpolation='none')
    plt.title("NN:{}, Data:{}".format(pred[i], target[i]))
    plt.xticks([])
    plt.yticks([])
  fig

In [ ]:
debug()

In [ ]:
for i in range(n_epochs):
    if i == 0: test(i)
    train()
    test(i+1)
debug()

In [ ]:
#ConvNet
class Net(nn.Module):
    def __init__(self, m_dim):
        super(Net, self).__init__()
        self.m_dim = m_dim
        self.conv1 = nn.Conv2d(1, m_dim, kernel_size=5)
        self.conv2 = nn.Conv2d(m_dim, m_dim*2, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(4*4*2*m_dim, 4*4*2*m_dim)
        self.fc2 = nn.Linear(4*4*2*m_dim, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 4*4*2*self.m_dim)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

network = Net(model_dimension)
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
for i in range(n_epochs):
    if i == 0: test(i)
    train()
    test(i+1)
debug()